In [3]:
### homework 4
#import套件 使用 as 代表別名，可以少打一點字
'''
(a)	Generate additive white Gaussian noise (amplitude of 10 and 30)
(b)	Generate salt-and-pepper noise ( probability 0.1 and 0.05)
(c)	Run box filter (3X3, 5X5) on all noisy images generated by (a)(b) 
(d)	Run median filter (3X3, 5X5) on all noisy images generated by (a)(b)
'''
import numpy as np
import cv2

def add_GaussianNoise(img,amp):
    # np.random.normal()
    # Draw random samples from a normal (Gaussian) distribution.
    noise=np.random.normal(loc=0, scale=1, size=img.shape)
    newImage=img+ amp*noise #additive   
    return newImage


def saltandpepperNoise(img,prob):
    '''
    img_src = cv2.imread('image.png')
    img_clone = img_src.copy()
    '''
    newImage=img.copy()
    '''
    p(x) = 1\(b - a)
    any value within the given interval is equally likely to be drawn by uniform.
    '''
    noise = np.random.uniform(low=0, high=1, size = img.shape)
    '''
    numpy.ndindex(*shape)
    An N-dimensional iterator object to index arrays.
    '''
    #for i,j in np.asarray(np.ndindex(noise.shape)):
    for i,j in np.ndindex(noise.shape):
        if noise[i,j]<prob:
            newImage[i,j]=0
        elif noise[i,j]>1-prob:
            newImage[i,j]=255
        else:
            pass        
    return newImage

##Box filtering = Averaging blurring 平均模糊
def boxfilter(img,size):
    #設定參數
    img_row,img_column=img.shape
    kernal_row=kernal_column=size
    # numpy.full(shape, fill_value, dtype=None, order='C', *, like=None))
    # 可以設定一個n*m維內部有相同的特定值或者是特定形態的
    box=np.full((size,size),1,dtype=int)   # 依照boxsize製作一個1的box(filter)
    newImage=np.zeros((img_row,img_column),np.int)
    
    # 計算kernel中心距離邊界有多遠，為的是擴大原始圖檔，方便後續迴圈處理
    dis_row, dis_column=int((kernal_row-1)/2),int((kernal_column-1)/2)
    
    #根據上述計算，製作一個比原始圖檔大的暫存圖檔，暫存圖檔為往上、往下、往左、往右分別外擴兩列/行，
    #外擴新增的pixel值=0，中間則就是原本輸入圖檔的值
    temp_img = np.full((int(img_row+ dis_row * 2),int(img_column+ dis_column * 2)),(-1))
    temp_img[dis_row:dis_row +img_row , dis_column: dis_column+img_column]=img.copy()
    #temp_img[row_dist:img_rows + row_dist, column_dist:img_columns + column_dist] = img.copy()
    '''
    1.numpy.unique(ar, return_index=False, return_inverse=False, return_counts=False, axis=None)
      函数是去除数组中的重复数字，并进行排序之后输出。
    2.在 Python 中若要將兩個 list 以迴圈的方式一次各取一個元素出來處理，可以使用 zip 打包之後配合 for 迴圈來處理
      假設我們有兩個 lists 分別存放鍵與值的資料：若要用這兩個 lists 建立一個 dict:dict(zip(keys, values))
    '''
    for i in range(dis_row,dis_row+img_row):
        for j in range(dis_column,dis_column+img_column):
            temp=temp_img[i-dis_row:i+dis_row+1,j-dis_column:j+dis_column+1]
            unique,count=np.unique(temp,return_counts=True)
            dict_len= dict(zip(unique,count))
            if (-1) in dict_len:
                temp2=temp.copy()
                for i_s in range(size):
                    for j_s in range(size):
                        if temp[i_s,j_s]==-1:
                            temp2[i_s,j_s]=0
                totalsum=np.sum(np.multiply(temp2,box))
                totalsize=size**2-dict_len[-1] #return -1的value(也就是個數)
            else:
                totalsum=np.sum(np.multiply(temp,box))
                totalsize=size**2 #**代表平方
            newImage[i-dis_row,j-dis_column]=totalsum/totalsize
    return newImage


# Median Filter，此種模糊化的方法也經常應用於相片的除噪（salt-and-pepper noise）
# Median Filter所使用的那個點是個既存的像素而非計算出來的像素，因此這是它可以被應用在除噪功能上的原因。
def medianfilter(img,size):
    img_row,img_column=img.shape
    kernal_row=kernal_column=size
    dis_row,dis_column=((kernal_row-1)/2,(kernal_column-1)/2)
    
    box=np.full((size,size),1,np.int)
    newImage=np.zeros((img_row,img_column),np.int)
    
    temp_img=np.full((img_row+ 2*dis_row ,img_column+ 2*dis_column),(-1))
    temp_img[dis_row:dis_row+img_row,dis_column:dis_column+img_column]=img
    
    for i in range(dis_row,dis_row+img_row):
        for j in range(dis_column,dis_column+img_column):
            temp=temp_img[i-dis_row:i+dis_row+1,j-dis_column:j+dis_column+1]
            unique,counts=np.uniuqe(temp,return_count=True)
            dict_len=dict(zip(unique,counts))# 拿來判斷用的
            if (-1) in dict_len:
                temp2=np.array([])
                for i_s in range(size):
                    for j_s in range(size):
                        if temp[i_s,j_s]==(-1):
                            pass
                        else:
                            temp2=np.append(temp2,temp[i_s,j_s])
                sort_dict_len=np.sort(temp2,axis=None)
                newImage[i-dis_row:j-dis_column]=sort_dict_len[int(len(sort_dict_len)/2)]
            else:
                sort_dict_len=np.sort(temp,axis=None)
                newImage[i-dis_row:j-dis_column]=sort_dict_len[int(len(sort_dict_len)/2)]    
    return newImage



## 開始自己得程式
Image=cv2.imread("lena.bmp",0)

# additive white Gaussian noise (amplitude of 10 and 30)
cv2.imwrite("GaussianNoise_10.bmp",add_GaussianNoise(Image,10))
cv2.imwrite("GaussianNoise_30.bmp",add_GaussianNoise(Image,30))

# salt-and-pepper noise ( probability 0.1 and 0.05)
cv2.imwrite("saltandpepperNoise_01.bmp",saltandpepperNoise(Image,0.1))
cv2.imwrite("saltandpepperNoise_002.bmp",saltandpepperNoise(Image,0.05))

# box filter (3X3, 5X5)
box3=boxfilter(Image,3)
cv2.imwrite("boxfilter_3x3.bmp",box3)
box5=boxfilter(Image,5)
cv2.imwrite("boxfilter_5x5.bmp",box5)
box7=boxfilter(Image,7)
cv2.imwrite("boxfilter_7x7.bmp",box7)

# median filter (3X3, 5X5)
cv2.imwrite("meidanfilter_3x3.bmp",medianfilter(Image,3))
cv2.imwrite("meidanfilter_5x5.bmp",medianfilter(Image,5))



TypeError: 'float' object cannot be interpreted as an integer